In [ ]:
!pip install sacrebleu sentencepiece &> /dev/null
!pip install nltk==3.2.4
!pip install transformers[sentencepiece] datasets &> /dev/null
!pip install sacrebleu sentencepiece &> /dev/null
!pip install huggingface_hub &> /dev/null
!pip install datasets transformers[sentencepiece] sacrebleu &> /dev/null
!pip install telepot &> /dev/null
!pip install evaluate

In [ ]:
!rm ita-eng.zip &> /dev/null
!rm rm -rf dataset &> /dev/null
!wget "https://www.manythings.org/anki/ita-eng.zip" &> /dev/null
!unzip "ita-eng.zip" -d "dataset" &> /dev/null

text_file = "dataset/ita.txt"

In [ ]:
import csv
import platform
import random
import time
import warnings

import datasets
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AutoTokenizer
import numpy as np
from scipy import spatial

warnings.filterwarnings("ignore")

##############################################################################
RANDOM_SEARCH = True # if false, the model will be trained using the following hyperparameters:

BATCH_SIZE = 128
LEARNING_RATE = 1e-05
WEIGHT_DECAY = 1e-06
EPOCHS_ = 1
##############################################################################
token = "5597879510:AAH1FSuZa7lA_xoAp-7JiUDE0OY38p-Tq5M"
theUrl = "https://api.telegram.org/bot" + token + "/sendMessage"
who = "-606080513"

random_search_trial = 3 # Not used if RANDOM_SEARCH is False
epochs = 5              # Not used if RANDOM_SEARCH is False

num_of_test = 30  # Number of sentences in the test set used to calculate the metric
text_file = "dataset/ita.txt"
prefix = "translate English to Italian: "
file_name = 'full_t5_result.csv'
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "it"
##############################################################################

pc_name = platform.node()

def sendMessage(text=""):
    import json
    import requests
    data = {'chat_id': who, 'disable_notification': 'false', 'text': text}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    #requests.post(theUrl, data=json.dumps(data, ensure_ascii=True), headers=headers)

def sendFile(file):
    import telepot
    f = open(file, 'rb')
    bot = telepot.Bot(token)
    bot.sendDocument(who, f)
    
if(RANDOM_SEARCH):
    true_epochs = epochs
else:
    true_epochs = EPOCHS_

text = """
>>> Starting T5... ["""+pc_name+"""]
Params:
is a random search?: """+str(RANDOM_SEARCH)+"""
epochs: """+str(true_epochs)+"""
num_of_test: """+str(num_of_test)+"""
random_search_trial: """+str(random_search_trial)+"""
"""
sendMessage(text)
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, ita, _copyright = line.split("\t")
    text_pairs.append({"translation": {"en": eng, "it": ita}})

# Now, let's split the sentence pairs into a training set, a validation set, and a test set.
random.seed(42)
#random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total sentences\n---\n")
print(f"{len(train_pairs)} training sentences")
print(f"{len(val_pairs)} validation sentences")
print(f"{len(test_pairs)} test sentences\n---\n")

raw_datasets = datasets.DatasetDict({"train": datasets.Dataset.from_pandas(pd.DataFrame(data=train_pairs)),
                                     "validation": datasets.Dataset.from_pandas(pd.DataFrame(data=val_pairs)),
                                     "test": datasets.Dataset.from_pandas(pd.DataFrame(data=test_pairs))})

model_checkpoint = "t5-small"  ## "t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"
model_name = model_checkpoint
print(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

def decode_sequence(model, sentence):
    inputs = tokenizer(prefix + sentence, return_tensors="tf").input_ids
    outputs = model.generate(inputs)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

def metric(target,output):
    import warnings
    warnings.filterwarnings("ignore")
    reference = [target.split(" ")]
    candidate = output.split(" ")
    score = sentence_bleu(reference, candidate)
    return score

def calculate_metric_on_testset(modello):
    result = 0
    count = 0
    for item in test_pairs[:num_of_test]:
        input_eng = item["translation"]["en"]  # eng
        target = item["translation"]["it"]   # ita
        output = decode_sequence(modello, input_eng)
        item_result = metric(target, output)
        result += item_result
        count += 1

    final_result = result / count
    return final_result

def append_to_csv(lista):
    with open(file_name, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(lista)

def one_single_trial(model, batch_size, epochs, learning_rate, weight_decay):
    error = False
    data_collator = DataCollatorForSeq2Seq(tokenizer, return_tensors="tf")
    train_dataset = tokenized_datasets["train"].to_tf_dataset(
        batch_size=batch_size,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=True,
        collate_fn=data_collator, )

    validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
        batch_size=batch_size,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator, )

    optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
    model.compile(optimizer=optimizer)  # To try: , loss="sparse_categorical_crossentropy")

    try:
        start_time = time.time()
        model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)
        end_time = time.time() - start_time

        start_time = time.time()
        metric_result = calculate_metric_on_testset(model)
        metric_end_time = time.time() - start_time
    except:
        end_time = -1
        metric_end_time = -1
        metric_result = -1
        error = True

    lista = [str(batch_size), str(learning_rate), str(weight_decay), str(epochs), str(end_time),
             str(metric_end_time), str(metric_result), str(error)]
    append_to_csv(lista)
    metric_end_time += end_time
    return model, metric_result, lista, end_time

def train():
    global random_search_trial
    global epochs

    total_time = 0
    model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
    print(model.summary())
    append_to_csv(['batch_size', 'learning_rate', 'weight_decay', 'epochs', 'training_time', 'metric_time', 'metric', 'error'])
    models = []

    if not RANDOM_SEARCH:
        random_search_trial = 1

    for _ in range(random_search_trial):
        if RANDOM_SEARCH:
            batch_size = random.choice([16,32,64])
            learning_rate = random.choice([0.00001, 0.000001, 1e-06, 1e-07])
            weight_decay = random.choice([0.00001, 0.000001, 1e-06, 1e-07])
        else:
            batch_size = BATCH_SIZE
            learning_rate = LEARNING_RATE
            weight_decay = WEIGHT_DECAY
            epochs = EPOCHS_

        print("Parameter selected: batch_size=", batch_size,
              ", learning_rate=", learning_rate,
              ", weight_decay=", weight_decay)

        model, metric_result, lista, end_time = one_single_trial(model, batch_size, epochs, learning_rate, weight_decay)
        models.append((model, metric_result, lista))

    model, best_result, hyperparams = models[np.argmax([y for x, y, z in models])]

    test_sentence = "So long and thanks for all the fish"
    translated_test_sentence = decode_sequence(model, test_sentence)

    test_sentence_2 = "Space is big. You just won't believe how vastly, hugely, mind-bogglingly big it is."
    translated_test_sentence_2 = decode_sequence(model, test_sentence_2)

    test_sentence_3 = test_pairs[71]["translation"]["en"]
    translated_test_sentence_3 = decode_sequence(model, test_sentence_3)

    text = """
💻 Worker: """+pc_name+"""
✨ Finished training ✨
🧠 Model: """+model_checkpoint+"""
🕒 Total time: """ + str(round(total_time, 6)) + """
📈 Best result: """ + str(round(best_result, 6)) + """
🔍 Params """ + str(hyperparams) + """
\n
Test sentence 1:
🇬🇧 : """ + test_sentence + """
🇮🇹 : """ + translated_test_sentence + """
\n
Test sentence 2:
🇬🇧 : """ + test_sentence_2 + """
🇮🇹 : """ + translated_test_sentence_2 + """\n
Test sentence 3:
🇬🇧 : """ + test_sentence_3 + """
🇮🇹 : """ + translated_test_sentence_3 + """
"""
    sendMessage(text)
    sendFile(file_name)
    return model

model = train()

In [ ]:
def calculate_metric_on_testset(modello):
    result = 0
    count = 0
    for item in test_pairs[:num_of_test]:
        input_eng = item["translation"]["en"]  # eng
        target = item["translation"]["it"]   # ita
        output = decode_sequence(modello, input_eng)
        item_result = metric(target, output)
        result += item_result
        count += 1

    final_result = result / count
    return final_result

In [ ]:
calculate_metric_on_testset(model)

In [ ]:
# Export
import tensorflow as tf
import os
model.save_weights('save/')

!zip -r method2_t5.zip save

!split -b 45m method2_t5.zip method2_t5.zip.pt_

for file in os.listdir(os.getcwd()):
        if file.startswith("method2_t5.zip.pt_"):
            sendFile(file)